## 1\. Introdução

### **1.1. TLDR**

 - **Dashboard**:
  - Google Data Studio (`link`).
 - **Processamento**:
  - Kaggle Notebook (`link`).
 - **Fontes**:
  - Casos pela universidade John Hopkins ([link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports));
  - Vacinação pela universidade de Oxford ([link](https://covid.ourworldindata.org/data/owid-covid-data.csv)).

### **1.2. Pandemia Coronavírus 2019**

> A COVID-19 é uma infecção respiratória aguda causada pelo coronavírus SARS-CoV-2, potencialmente grave, de elevada transmissibilidade e de distribuição global. Fonte: Governo brasileiro ([link](https://www.gov.br/saude/pt-br/coronavirus/o-que-e-o-coronavirus)).

A disponibilidade de dados sobre a evolução da pandemia no tempo em uma determinada região geográfica é fundamental para o seu combate! Este projeto busca construir um dashboard de dados para exploração e visualização interativa de dados sobre o avanço de casos e da vacinação do Brasil. O processamento de dados está neste `link` e o dashboard, neste `link`.

### **1.3. Dados**

Os dados sobre **casos da COVID-19** são compilados pelo centro de ciência de sistemas e engenharia da universidade americana **John Hopkins** ([link](https://www.jhu.edu)). Os dados são atualizados diariamente deste janeiro de 2020 com uma granularidade temporal de dias e geográfica de regiões de países (estados, condados, etc.). O website do projeto pode ser acessado neste [link](https://systems.jhu.edu/research/public-health/ncov/) enquanto os dados, neste [link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports). Abaixo estão descritos os dados derivados do seu processamento.

 - **date**: data de referência;
 - **state**: estado;
 - **country**: país;
 - **population**: população estimada;
 - **confirmed**: número acumulado de infectados;
 - **confirmed_1d**: número diário de infectados;
 - **confirmed_moving_avg_7d**: média móvel de 7 dias do número diário de infectados;
 - **confirmed_moving_avg_7d_rate_14d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **deaths**: número acumulado de mortos;
 - **deaths_1d**: número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias do número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **month**: mês de referência;
 - **year**: ano de referência.

Os dados sobre **vacinação da COVID-19** são compilados pelo projeto Nosso Mundo em Dados (*Our World in Data* ou OWID) da universidade britânica de **Oxford** ([link](https://www.ox.ac.uk)). Os dados são **atualizados diariamente** deste janeiro de 2020 com uma **granularidade temporal de dias e geográfica de países**. O website do projeto pode ser acessado neste [link](https://ourworldindata.org) enquanto os dados, neste [link](https://covid.ourworldindata.org/data/owid-covid-data.csv). Abaixo estão descritos os dados derivados do seu processamento.

 - **date**: data de referência;
 - **country**: país;
 - **population**: população estimada;
 - **total**: número acumulado de doses administradas;
 - **one_shot**: número acumulado de pessoas com uma dose;
 - **one_shot_perc**: número acumulado relativo de pessoas com uma dose;
 - **two_shots**: número acumulado de pessoas com duas doses;
 - **two_shot_perc**: número acumulado relativo de pessoas com duas doses;
 - **three_shots**: número acumulado de pessoas com três doses;
 - **three_shot_perc**: número acumulado relativo de pessoas com três doses;
 - **month**: mês de referência;
 - **year**: ano de referência.

## 2\. Análise Exploratória de Dados

Nesta sessão vamos utilizar os seguintes pacotes Python para processar os dados bruto em um formato adequado para um painel para exploração interativa de dados.

In [1]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

### **2.1. Casos**

Vamos processar os dados de **casos** da universidade John Hopkins. Os dados estão organizados em um arquivo por dia. Assim sendo, Será necessário interar dentro de um intervalo de tempo para extrair-los



In [2]:
def intervalo_tempo(inicio: datetime, final: datetime) -> Iterator[datetime]:
  dias: int = (final - inicio).days
  for dia in range(dias):
    yield inicio + timedelta(dia)

In [3]:
data_inicio = datetime(2020, 12, 11)
data_final = datetime(2022, 1, 1)

**Observação**
****
Repare que, embora o intervalo a ser analisado seja o ano de 2021, incluí na análise os 21 dias finais do ano de 2020.

In [4]:
cases = None
cases_is_empty = True

for date in intervalo_tempo(inicio = data_inicio, final = data_final):

  date_str = date.strftime('%m-%d-%Y')
  url_fonte = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  caso = pd.read_csv(url_fonte, sep=',')
  caso = caso.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered',
                    'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
  caso = caso.query('Country_Region == "Brazil"').reset_index(drop=True)
  caso['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))

  if cases_is_empty:
    cases = caso
    cases_is_empty = False
  else:
    cases = pd.concat([cases, caso], ignore_index=True)

#### **2.1.1. Wrangling**

Vamos manipular os dados para o dashboard. O foco é em garantir uma boa granularidade e qualidade da base de dados. Vamos ajustar o nome das colunas padronizadas em minúsculas e utilizando nomes mais simples. Também modificaremos o nome dos estados brasileiros incluindo o ascento gráfico para poder utilizar a ferramenta de georefferenciamento do google data studio. Por fim, vamos adicionar algumas colunas contendo chaves de mês e ano e adicionar colunas com estimativa populacional e métricas como média móvel e estabilidade.

In [5]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10422 entries, 0 to 10421
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province_State  10422 non-null  object        
 1   Country_Region  10422 non-null  object        
 2   Confirmed       10422 non-null  int64         
 3   Deaths          10422 non-null  int64         
 4   Incident_Rate   10422 non-null  float64       
 5   Date            10422 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 488.7+ KB


In [6]:
cases = cases.rename(columns={'Province_State': 'state', 'Country_Region': 'country'})

for coluna in cases.columns:
  cases = cases.rename(columns={coluna: coluna.lower()})

states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)
cases['month'] = cases['date'].apply(lambda x: x.strftime('%Y-%m'))
cases['year'] = cases['date'].apply(lambda x: x.strftime('%Y'))
cases['population'] = round(100000 * (cases['confirmed']/cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)

In [7]:

cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate <= 0.85:
    status = 'downward'
  elif rate >= 1.15:
    status = 'upward'
  else:
    status = 'stable'

  return status

for state in cases['state'].drop_duplicates():

  cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values(by=['date'])

  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
  cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

  if cases_is_empty:
    cases_ = cases_per_state
    cases_is_empty = False
  else:
    cases_ = pd.concat([cases_, cases_per_state], ignore_index=True)


In [8]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10422 entries, 0 to 10421
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   state       10422 non-null  object        
 1   country     10422 non-null  object        
 2   confirmed   10422 non-null  int64         
 3   deaths      10422 non-null  int64         
 4   date        10422 non-null  datetime64[ns]
 5   month       10422 non-null  object        
 6   year        10422 non-null  object        
 7   population  10422 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 651.5+ KB


In [9]:
cases = cases_

In [10]:
# Fazendo um type casting para as novas colunas

cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

In [11]:
# Reorganizando as colunas

cases = cases[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d',
               'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d',
               'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d',
               'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]

In [12]:
# Selecionando os dados de 2021

cases = cases.query('year == "2021"').reset_index(drop=True)

In [ ]:
cases.tail(n=31)

In [14]:
# Criando o arquivo CSV para ser utilizado no Google Data Studio

cases.to_csv('./covid-cases.csv', sep=',', index=False)

### **2.2. Vacinação**

Os dados de **vacinação** terão como fonte os dados obtidos da universidade de Oxford. Desta vez, teremos as informações de interesse armazenadas em um único arquivo, o que facilitará nosso processo. Apenas iremos selecionar os dados de interesse e manipular para que tenham uma granularidade adequada.  

In [ ]:
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

In [ ]:
vaccines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1336 entries, 0 to 1335
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   location                 1336 non-null   object        
 1   population               1336 non-null   float64       
 2   total_vaccinations       695 non-null    float64       
 3   people_vaccinated        691 non-null    float64       
 4   people_fully_vaccinated  675 non-null    float64       
 5   total_boosters           455 non-null    float64       
 6   date                     1336 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 73.2+ KB


**Observação**
****
Quando analisamos os dados obtidos, percebemos que algumas colunas possuem dados faltantes, todos eles relacionados ao número de vacinados. Para tratar desses valores nulos, vamos preenche-los com o último valor válido. Como se trata de um número acumulativo, não devemos ter problemas.

In [ ]:
vaccines = vaccines.fillna(method='ffill')

In [ ]:
# Agora possuímos valores mais consistentes
vaccines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1336 entries, 0 to 1335
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   location                 1336 non-null   object        
 1   population               1336 non-null   float64       
 2   total_vaccinations       956 non-null    float64       
 3   people_vaccinated        956 non-null    float64       
 4   people_fully_vaccinated  937 non-null    float64       
 5   total_boosters           728 non-null    float64       
 6   date                     1336 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 73.2+ KB


In [ ]:
# Vamos selecionar o período de interesse, renomear as colunas e adicionar outras
# colunas de interesses, como as de chave temporal e as que contém dados relativos.

vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

vaccines = vaccines.rename(columns={'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots'})

vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))

vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

In [ ]:
# Agora vamos realizar o type casting

vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

In [ ]:
# Finalmente, vamos reorganizar as colunas

vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot',
                     'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots',
                     'three_shots_perc', 'month', 'year']]

In [ ]:
vaccines.head(n=20)

In [ ]:
vaccines.tail(n=20)

In [ ]:
# Carregando  os dados relativos a vacinação do Brasil

vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)

###**2.2.1** Dados Extras

In [ ]:
# Agora vamos repetir o processo de extração de dados de vacinação com outros países sul americanos

vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

vaccines = vaccines.query('location == "Argentina"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

vaccines = vaccines.fillna(method='ffill')

vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

vaccines = vaccines.rename(columns={'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots'})

vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))

vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot',
                     'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots',
                     'three_shots_perc', 'month', 'year']]

vaccines.to_csv('./argentina-vaccines.csv', sep=',', index=False)

In [ ]:

vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

vaccines = vaccines.query('location == "Chile"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

vaccines = vaccines.fillna(method='ffill')

vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

vaccines = vaccines.rename(columns={'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots'})

vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))

vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot',
                     'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots',
                     'three_shots_perc', 'month', 'year']]

vaccines.to_csv('./chile-vaccines.csv', sep=',', index=False)